<a href="https://colab.research.google.com/github/shjoo0407/political_disposition/blob/main/eun/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [1]:
import io
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
import numpy as np
import re, string

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/2023/sw공모전/data

/content/drive/MyDrive/2023/sw공모전/data


In [5]:
ls

'df_summarization(2022)'   국힘2022.csv      문재인_국힘.csv   박근혜_국힘.csv
 NPP_final_0729.csv        노무현.csv        민주당2020.csv    이명박.csv
 TMP_final_0729.csv        노무현_국힘.csv   민주당2022.csv    이명박_국힘.csv
 국힘2020.csv              문재인.csv        박근혜.csv


In [4]:
df1 = pd.read_csv('민주당2022.csv')
df2 = pd.read_csv('국힘2022.csv')
df3 = pd.read_csv('민주당2020.csv')
df4 = pd.read_csv('국힘2020.csv')

In [5]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(4663, 6)
(4925, 6)
(3558, 6)
(5273, 6)


In [6]:
df1['target'] = '민주당'
df2['target'] = '국민의힘'
df3['target'] = '민주당'
df4['target'] = '국민의힘'

df_1 = pd.concat([df1, df2]).reset_index()
df_2 = pd.concat([df3, df4]).reset_index()

df = pd.concat([df_1, df_2]).reset_index()

In [7]:
df.head(3)

,level_0,index,Unnamed: 0,title,year,quarter,content,summary,target,category
0,0,0,0,"[박성준 대변인] 정체불명의 해외발 우편물, 국민 불안이 확산되는데 정부는 손놓고 ...",2023,3,"정체불명의 해외발 우편물, 국민 불안이 확산되는데 정부는 손놓고 있습니까 3일째 전...",전국 곳곳에 해외에서 발송된 정체불명의 우편물들이 1000건 이상 신고되면서 국민 ...,민주당,NaN
1,1,1,1,[박성준 대변인] 검찰은 조작 수사도 모자라 이제는 진술을 왜곡해 언론에 퍼뜨려 여...,2023,3,검찰은 조작 수사도 모자라 이제는 진술을 왜곡해 언론에 퍼뜨려 여론을 호도하고 있습...,검찰은 조작 수사도 모자라 진술을 왜곡해 언론에 퍼뜨려 여론을 호도하고 있으며 이 ...,민주당,NaN
2,2,2,2,[박성준 대변인] 윤석열 대통령은 높아지는 군사적 긴장에 언제까지 팔짱만 끼고 있을...,2023,3,윤석열 대통령은 높아지는 군사적 긴장에 언제까지 팔짱만 끼고 있을 겁니까 북한이 오...,북한이 서해상으로 순항미사일 여러 발을 발사한 것은 북한의 거듭되는 무력 도발과 위...,민주당,NaN


In [8]:
df = df[['summary','content','target']]

In [9]:
df['target'].value_counts()

국민의힘    10198
민주당      8221
Name: target, dtype: int64

In [10]:
df.isna().sum()

summary    0
content    0
target     0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
df['target_encoded'] = label_encoder.fit_transform(df['target'])

df.head()

,summary,content,target,target_encoded
0,전국 곳곳에 해외에서 발송된 정체불명의 우편물들이 1000건 이상 신고되면서 국민 ...,"정체불명의 해외발 우편물, 국민 불안이 확산되는데 정부는 손놓고 있습니까 3일째 전...",민주당,1
1,검찰은 조작 수사도 모자라 진술을 왜곡해 언론에 퍼뜨려 여론을 호도하고 있으며 이 ...,검찰은 조작 수사도 모자라 이제는 진술을 왜곡해 언론에 퍼뜨려 여론을 호도하고 있습...,민주당,1
2,북한이 서해상으로 순항미사일 여러 발을 발사한 것은 북한의 거듭되는 무력 도발과 위...,윤석열 대통령은 높아지는 군사적 긴장에 언제까지 팔짱만 끼고 있을 겁니까 북한이 오...,민주당,1
3,국정 운영 방향을 남 탓으로 정한 정권은 없었고 윤 정권은 어떤 국정 비전도 보여주...,국정 운영 방향을 남 탓으로 정한 정권은 윤석열 정권뿐입니다 “윤 대통령의 국정 운...,민주당,1
4,채 상병은 기본적인 안전장비도 지급받지 못한 채 임무에 투입되었고 뜨거운 애국심으로...,"누구보다 나라를 사랑했던 청년, 명령에 충실했던 병사를 기억하겠습니다 경북 예천에서...",민주당,1


In [12]:
df['target_encoded'].unique()

array([1, 0])

### Text Classification


In [13]:
import tensorflow as tf

In [14]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


In [15]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm


In [16]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [17]:
# import numpy as np
# from sklearn.model_selection import train_test_split


# X_train, X_test, y_train, y_test = train_test_split(df['content'], df['target_encoded'], test_size=0.2, random_state=42, shuffle=True)

# print('Number of training reviews: {}'.format(len(X_train)))
# print('Number of testing reviews: {}'.format(len(X_test)))

Number of training reviews: 14735
Number of testing reviews: 3684


In [19]:
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(df['content'], df['target_encoded'], test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

print('Number of training reviews: {}'.format(len(X_train)))
print('Number of testing reviews: {}'.format(len(X_test)))
print('Number of validation reviews: {}'.format(len(X_val)))

Number of training reviews: 14735
Number of testing reviews: 1842
Number of validation reviews: 1842


In [20]:
max_len = 512
vocab_size = 20000  # 빈도수 상위 2만개의 단어만 사용

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Convert text to integer sequences
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)


# Pad sequences
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)
X_val_padded = pad_sequences(X_val_seq, maxlen=max_len)

print('Shape of X_train_padded:', X_train_padded.shape)
print('Shape of X_test_padded:', X_test_padded.shape)
print('Shape of X_val_padded:', X_val_padded.shape)

Shape of X_train_padded: (14735, 512)
Shape of X_test_padded: (1842, 512)
Shape of X_val_padded: (1842, 512)


In [27]:
embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
num_heads = 2 # 어텐션 헤드의 수
dff = 64  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train_padded, y_train, batch_size=256, epochs=50, validation_data=(X_test_padded, y_test))

In [28]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_padded, y_train, batch_size=256, epochs=50,  validation_data=(X_val_padded, y_val), callbacks=[early_stopping])

Epoch 1/50
58/58 [==============================] - 75s 287ms/step - loss: 0.7113 - accuracy: 0.5420 - val_loss: 0.6599 - val_accuracy: 0.5565
Epoch 2/50
58/58 [==============================] - 15s 267ms/step - loss: 0.5264 - accuracy: 0.7177 - val_loss: 0.2087 - val_accuracy: 0.9202
Epoch 3/50
58/58 [==============================] - 14s 237ms/step - loss: 0.1840 - accuracy: 0.9417 - val_loss: 0.1148 - val_accuracy: 0.9598
Epoch 4/50
58/58 [==============================] - 14s 244ms/step - loss: 0.0792 - accuracy: 0.9789 - val_loss: 0.1097 - val_accuracy: 0.9647
Epoch 5/50
58/58 [==============================] - 13s 229ms/step - loss: 0.0471 - accuracy: 0.9881 - val_loss: 0.1161 - val_accuracy: 0.9636
Epoch 6/50
58/58 [==============================] - 13s 232ms/step - loss: 0.0187 - accuracy: 0.9966 - val_loss: 0.1700 - val_accuracy: 0.9631
Epoch 7/50
58/58 [==============================] - 12s 217ms/step - loss: 0.0097 - accuracy: 0.9985 - val_loss: 0.1930 - val_accuracy: 0.9631

In [29]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))

58/58 [==============================] - 0s 8ms/step - loss: 0.0945 - accuracy: 0.9707
테스트 정확도: 0.9707


### 형태소

In [ ]:
! pip install konlpy

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

In [ ]:
#불용어 사전 만들기
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

#### 형태소 분석기 비교

In [ ]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [ ]:
text = df['split_content2'][0]

In [ ]:
def extract_nouns_verbs(text, analyzer):
    pos = analyzer.pos(text)

    nouns = []  # 명사 저장할 리스트
    verbs = []  # 동사 저장할 리스트

    for word_tag in pos:
        word, tag = word_tag[0], word_tag[1]
        if tag.startswith("N"):  # 명사 태그
            nouns.append(word)
        elif tag.startswith("V"):  # 동사 태그
            verbs.append(word)

    return nouns, verbs


nouns_ko, verbs_ko = extract_nouns_verbs(text, komoran)
print("Komoran 명사:", nouns_ko)
print("Komoran 동사:", verbs_ko)

In [ ]:
def extract_nouns_verbs(text, analyzer):
    pos = analyzer.pos(text)

    nouns = []  # 명사 저장할 리스트
    verbs = []  # 동사 저장할 리스트

    for word, tag in pos:
        if tag.startswith("N"):  # 명사 태그
            nouns.append(word)
        elif tag.startswith("V"):  # 동사 태그
            verbs.append(word)

    return nouns, verbs

nouns_k, verbs_k = extract_nouns_verbs(text, kkma)
print("Kkma 명사:", nouns_k)
print("Kkma 동사:", verbs_k)

In [ ]:
def extract_nouns_verbs(text, analyzer):
    pos = analyzer.pos(text, ntags=22)

    nouns = []  # 명사 저장할 리스트
    verbs = []  # 동사 저장할 리스트

    for word, tag in pos:
        if tag.startswith("N"):  # 명사 태그
            nouns.append(word)
        elif tag.startswith("V"):  # 동사 태그
            verbs.append(word)

    return nouns, verbs

nouns_h, verbs_h = extract_nouns_verbs(text, hannanum)
print("Hannanum 명사:", nouns_h)
print("Hannanum 동사:", verbs_h)

In [ ]:
texts = df['split_content2']

data = []
for text in texts:
    nouns, verbs = extract_nouns_verbs(text, hannanum)
    data.append({'Text': text, 'Nouns': nouns, 'Verbs': verbs})

df['noun'] = pd.DataFrame(data)
df.head()

In [ ]:
df.to_csv('df.csv')